In [1]:
import pathlib

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tifffile as tf  # write tiff files
from cytocherrypick.calculations import find_median
from PIL import Image  # read tiff files
from toml import load
from tqdm import tqdm  # progress bar

In [2]:
CELL_TYPE = "PBMC"

In [3]:
path = pathlib.Path(f"../results/{CELL_TYPE}/single_cell_predictions.parquet").resolve()

# read in the data
df = pd.read_parquet(path)
# show all columns
pd.set_option("display.max_columns", None)
df.head()

,oneb_Metadata_Treatment_Dose_Inhibitor_Dose,Metadata_Well,Metadata_Nuclei_Location_Center_Y,Metadata_Nuclei_Location_Center_X,Metadata_Image_Metadata_Site,group,Cytoplasm_RadialDistribution_ZernikeMagnitude_CorrER_9_9,Nuclei_RadialDistribution_ZernikePhase_CorrDNA_7_1,Cells_RadialDistribution_ZernikeMagnitude_CorrPM_6_4,Metadata_ImageNumber,Metadata_Cells_Number_Object_Number,Metadata_Site,comparison,treatment,feature,image_compsite_path,image_composite_crop_path,image_DAPI_path,image_ER_path,image_GasderminD_path,image_AGP_path,image_Mitochondria_path,image_compsite_crop_path,image_DAPI_crop_path,image_ER_crop_path,image_GasderminD_crop_path,image_AGP_crop_path,image_Mitochondria_crop_path
0,DMSO_0.100_%_DMSO_0.025_%,B07,760.351675,1375.437799,9,Control,-0.402880,1.671473,0.834995,89.0,1679.0,9,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrDNA...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...
1,DMSO_0.100_%_Z-VAD-FMK_100.000_uM,K12,985.997753,242.062921,8,Control,-0.128377,1.671471,-0.758292,344.0,1596.0,8,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrDNA...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...
2,H2O2_100.000_nM_DMSO_0.025_%,H05,751.917676,1066.164649,12,Control,-0.451012,1.671471,-0.273077,60.0,1437.0,12,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrDNA...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...
3,DMSO_0.100_%_Z-VAD-FMK_30.000_uM,E12,2080.795367,131.328185,8,Control,0.038387,-1.701544,-0.643369,344.0,2893.0,8,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrDNA...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyroptosis_s...,/home/lippincm/Documents/4TB/data/pyrop

In [4]:
df.columns
image_path_columns = [
    "image_compsite_path",
    "image_DAPI_path",
    "image_ER_path",
    "image_GasderminD_path",
    "image_AGP_path",
    "image_Mitochondria_path",
    "image_composite_crop_path",
    "image_DAPI_crop_path",
    "image_ER_crop_path",
    "image_GasderminD_crop_path",
    "image_AGP_crop_path",
    "image_Mitochondria_crop_path",
]
for path_type in image_path_columns:
    # print(path_type)
    for image_path in df[path_type]:
        # open the image
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        # image to array
        image_array = np.array(image)
        resolution = 3.3446  # pixels per micrometer
        # get the image size
        image_size = image_array.shape
        print(image_size)
        if image_array.shape[1] <= 100:
            scale_bar_length = 5  # um
            scale_bar_height = 1  # pixels
            padding = 3  # pixels
        elif image_array.shape[1] > 100:
            scale_bar_length = 100
            scale_bar_height = 10
            padding = 10
        # get the bottom right most corner based on scale 1 % of pixels
        scale_bar_x = (
            image_size[1] - (scale_bar_length * resolution) - padding - padding
        )
        scale_bar_y = image_size[0] - (scale_bar_height) - padding
        print(scale_bar_x, scale_bar_y)
        # draw the scale bar
        new = cv2.rectangle(
            image_array,
            (int(scale_bar_x), int(scale_bar_y)),
            (image_size[1] - padding, image_size[0] - padding),
            (255, 255, 255),
            -1,
        )
        # show the image
        # plt.imshow(new)
        # plt.show()
        # save the image with the scale bar via cv2
        cv2.imwrite(image_path, new)
#

(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3)
1805.54 2140
(2160, 2160, 3

In [5]:
image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
print(image_path)
# image to array
print(image)
image_array = np.array(image)
resolution = 3.3446  # pixels per micrometer
# get the image size
image_size = image_array.shape
print(image_size)
if image_array.shape[1] <= 100:
    scale_bar_length = 5  # um
    scale_bar_height = 1  # pixels
    padding = 3  # pixels
elif image_array.shape[1] > 100:
    scale_bar_length = 100
    scale_bar_height = 10
    padding = 10
# get the bottom right most corner based on scale 1 % of pixels
scale_bar_x = (
    image_size[1] - (scale_bar_length * resolution) - padding - padding
)
scale_bar_y = image_size[0] - (scale_bar_height) - padding
print(scale_bar_x, scale_bar_y)
# draw the scale bar
new = cv2.rectangle(
    image_array,
    (int(scale_bar_x), int(scale_bar_y)),
    (image_size[1] - padding, image_size[0] - padding),
    (255, 255, 255),
    -1,
)
# show the image
# plt.imshow(new)
# plt.show()

/home/lippincm/Documents/4TB/data/pyroptosis_signature_data_analysis/figures/3.figure3/figures/images/PBMC/pyroptosis__apoptosis_pyroptosis__Cells_RadialDistribution_ZernikeMagnitude_CorrPM_6_4_magenta_channel_crop_cell_5.png
[[5722 6086 6043 ... 2222 2132 2050]
 [5585 5868 5817 ... 2266 2130 1960]
 [6061 5593 4892 ... 2363 2196 1939]
 ...
 [3381 3116 3232 ...  255 2726 2767]
 [3088 3147 3157 ... 2720 2885 2954]
 [3263 3247 3363 ... 2823 2857 3039]]
(100, 100)
77.277 96
